# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2015	East is not a ‘big bird’: The etymology of the star Altair in the Carolinian sidereal compass.	Oceanic Linguistics 54(2).579-588	About	Holton, Gary, Calistus Hachibmai, Ali Haleyalur, Jerry Lipka, and Donald Rubinstein. 2015. East is not a ‘big bird’: The etymology of the star Altair in the Carolinian sidereal compass. <i>Oceanic Linguistics</i> 54(2).579-588	http://doi.org/10.1353/ol.2015.0021
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf


## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [14]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,description,citation,url_slug,paper_url,pub_file,image
0,2015-12-01,East is not a ‘big bird’: The etymology of the...,Oceanic Linguistics,We argue that the Micronesian constellation ce...,"Holton, Gary, Calistus Hachibmai, Ali Haleyalu...",2015-big-bird,http://doi.org/10.1353/ol.2015.0021,NaN,NaN
1,2019-03-01,Language and Toponymy in Alaska and Beyond,University of Hawai‘i Press,It is difficult to imagine place names researc...,"Holton, Gary and Thomas F. Thornton. 2019. <i>...",2019-language-and-toponymy,https://nflrc.hawaii.edu/ldc/sp-17/,NaN,ldc-sp17-cover.png
2,2018-12-31,Reflections on Language Documentation 20 Years...,University of Hawai‘i Press,This volume reflects on key issues in the fiel...,"McDonnell, Bradley, Andrea Berez-Kroeker, and ...",2018-reflections-on-language-documentation,https://nflrc.hawaii.edu/ldc/sp-15/,NaN,ldc-sp15-cover.png
3,2017-03-30,A unified system of spatial orientation in the...,NUSA,The Austronesian and non-Austronesian language...,"Holton, Gary. 2017. A unified system of spatia...",2017-halmahera-orientation,http://hdl.handle.net/10108/89846,NaN,NaN
4,2017-01-01,The Papuan Languages of East Nusantara,The Languages and Linguistics of the New Guine...,NaN,"Holton, Gary and Marian Klamer. 2017. The Papu...",2017-papuan-east-nusantara,NaN,NaN,NaN
5,2018-01-01,Interdisciplinary language documentation,Oxford Handbook of Endangered Languages,NaN,"Holton, Gary. 2018. Interdisciplinary language...",2018-interdisciplinary-language-documentation,NaN,NaN,NaN
6,2020-04-01,Variation in spatial orientation systems acros...,Linguistics Vanguard,NaN,"Holton, Gary and Leah Pappas. in review. Varia...",2020-spatial-variation,NaN,NaN,NaN
7,2020-01-01,Spatial orientation in the Malayo-Polynesian l...,The Oxford Guide to the Malayo-Polynesian Lang...,NaN,"Pappas, Leah and Gary Holton. to appear. Spati...",2020-MP-space,NaN,NaN,NaN
8,2014-03-12,Linguistic phylogenies support back-migration ...,PLoS ONE,Recent arguments connecting Na-Dene languages ...,"Sicoli, Mark and Gary Holton. 2014. Linguistic...",2014-dy-phylogeny,http://doi.org/10.1371/journal.pone.0091722],NaN,neighbornet.png
9,2020-08-12,"People, ethics, and data",Open Handbook of Linguistic Data Management,NaN,"Holton, Gary, Wesley Y. Leonard, and Peter L. ...",2020-data-ethics,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [22]:
import os
for row, item in publications.iterrows():
    print(len(str(item.pub_file)))

3
3
3
3
3
3
3
3
3
3
3


In [25]:
import os
for row, item in publications.iterrows():

    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]

    ## YAML variables

    md = "---\ntitle: \""   + item.title + '"\n'

    md += """collection: publications"""

    md += """\npermalink: /publication/""" + item.url_slug # html_filename

    md += "\ndate: " + str(item.pub_date)

    md += "\nvenue: '" + html_escape(item.venue) + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"

    md += "\ncitation: '" + html_escape(item.citation) + "'"

    md += "\n---"

    ## Markdown description for individual page

    if len(str(item.description)) > 5:
        md += "\n\n"
        if len(str(item.image)) > 5:
            md = md + '<img src="/images/' + str(item.image) + '" width="200" align="left" style="margin-right: 15px;">'
        md +=  html_escape(item.description) + "\n"


    if len(str(item.pub_file)) > 5:
        md += "\n\n[Download](/files/" + item.pub_file + ")\n"

#    if len(str(item.paper_url)) > 5:
#        md += "\n\n[<a href='" + item.paper_url + "'>download</a>]\n"

    # md += "\nRecommended citation: " + item.citation

    md_filename = os.path.basename(md_filename)

    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [10]:
!ls ../_publications/

2014-03-12-2014-dy-phylogeny.md
2015-12-01-2015-big-bird.md
2017-01-01-2017-papuan-east-nusantara.md
2017-03-30-2017-halmahera-orientation.md
2018-01-01-2018-interdisciplinary-language-documentation.md
2018-12-31-2018-reflections-on-language-documentation.md
2019-03-01-2019-language-and-toponymy.md
2019-12-31-2020-sense-of-place.md
2020-01-01-2020-MP-space.md
2020-04-01-2020-spatial-variation.md
2020-08-12-2020-data-ethics.md


In [11]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
